In [8]:
import os

def read_input(filename):
    """
    Read the input file and parse the problem parameters.
    Returns: rows, cols, vehicles, number of rides, bonus, total steps, rides list
    """
    with open(filename, "r") as f:
        rows, cols, veh, n_rides, bonus, steps = map(int, f.readline().split())
        rides = []
        for _ in range(n_rides):
            a, b, x, y, s, f_end = map(int, f.readline().split())
            rides.append((a, b, x, y, s, f_end))
    return rows, cols, veh, n_rides, bonus, steps, rides


def manhattan(a, b):
    """
    Compute Manhattan distance between two points a and b.
    """
    return abs(a[0] - b[0]) + abs(a[1] - b[1])


def solve(rows, cols, veh, n_rides, bonus, steps, rides):
    """
    Greedy heuristic solver:
    Assign rides to vehicles based on earliest available vehicle
    and earliest feasible ride start time.
    """
    # Precompute the length of each ride
    ride_len = [manhattan((a, b), (x, y)) for (a, b, x, y, s, f) in rides]

    # Vehicle states
    t_of_the_car = [0] * veh              # current time for each vehicle
    car_x = [0] * veh                     # current row position of each vehicle
    car_y = [0] * veh                     # current column position of each vehicle
    sched = [[] for _ in range(veh)]      # rides assigned to each vehicle

    taken = [False] * n_rides             # ride assignment tracker
    score = 0

    while True:
        # Select the vehicle that becomes free the earliest
        cur_t = steps
        j = -1
        for v in range(veh):
            if t_of_the_car[v] < cur_t:
                cur_t = t_of_the_car[v]
                j = v

        # All vehicles are busy beyond simulation steps
        if cur_t >= steps or j == -1:
            break

        # Find the best feasible ride for this vehicle
        best_start = steps
        k = -1
        for i in range(n_rides):
            if taken[i]:
                continue
            a, b, x, y, s, f = rides[i]

            go = manhattan((car_x[j], car_y[j]), (a, b))  # distance to ride start
            finish = cur_t + go + ride_len[i]             # time to complete this ride

            if finish <= f:  # ride can be finished before its latest finish
                start_time = max(cur_t + go, s)          # wait until earliest start if needed
                if start_time < best_start:             # choose ride that starts earliest
                    best_start = start_time
                    k = i

        # No feasible ride found for this vehicle at this time
        if k == -1:
            t_of_the_car[j] = steps
            continue

        # Assign ride k to vehicle j
        taken[k] = True
        a, b, x, y, s, f = rides[k]

        sched[j].append(k)
        car_x[j], car_y[j] = x, y
        t_of_the_car[j] = best_start + ride_len[k]

        # Update score
        score += ride_len[k]
        if best_start == s:
            score += bonus

    return score, sched


def write_output(filename, sched):
    """
    Write the vehicle schedules to output file.
    """
    with open(filename, "w") as f:
        for rides in sched:
            f.write(str(len(rides)) + " " + " ".join(map(str, rides)) + "\n")

    # Print a summary of assignments
    print("=== Vehicle Ride Assignments ===")
    for i, rides in enumerate(sched):
        print(f"Vehicle {i} is assigned {len(rides)} rides: {rides}")


if __name__ == "__main__":
    filename = "./input/d.in"
    rows, cols, veh, n_rides, bonus, steps, rides = read_input(filename)

    score, sched = solve(rows, cols, veh, n_rides, bonus, steps, rides)
    print("Score =", score)

    os.makedirs("./output", exist_ok=True)
    base = os.path.basename(filename).replace(".in", ".out")
    outname = os.path.join("./output", base)
    write_output(outname, sched)
    print(f"\nOutput saved to {outname}")


Score = 11739569
=== Vehicle Ride Assignments ===
Vehicle 0 is assigned 11 rides: [2735, 8227, 1582, 6623, 9434, 9690, 8168, 1732, 5294, 8910, 6439]
Vehicle 1 is assigned 15 rides: [6689, 1227, 8713, 6879, 3635, 3328, 5355, 4831, 7408, 6520, 9883, 7416, 9765, 6795, 2845]
Vehicle 2 is assigned 21 rides: [3090, 6654, 661, 6711, 977, 6886, 9564, 5405, 2809, 5219, 536, 133, 5658, 684, 1525, 5191, 5728, 5767, 63, 8531, 1998]
Vehicle 3 is assigned 13 rides: [1209, 477, 4233, 4299, 4250, 9116, 3818, 952, 9028, 7207, 1417, 6190, 3224]
Vehicle 4 is assigned 17 rides: [6466, 7455, 2523, 4920, 3915, 8155, 8259, 2119, 3675, 396, 4247, 6677, 7924, 8428, 1395, 1626, 8364]
Vehicle 5 is assigned 30 rides: [5018, 3245, 4635, 9382, 7277, 6182, 1762, 1746, 514, 5702, 4227, 138, 6868, 7610, 2340, 9506, 4369, 7735, 547, 5680, 8828, 2201, 3350, 1824, 5512, 5109, 9277, 4779, 5547, 4592]
Vehicle 6 is assigned 34 rides: [9313, 3294, 9666, 1200, 4558, 3319, 5495, 6313, 4331, 3531, 9248, 6156, 2906, 7592, 9300, 

In [4]:
import os
import bisect

def read_input(filename):
    """
    Read the input file and parse the problem parameters.
    Returns: rows, cols, vehicles, number of rides, bonus, total steps, rides list
    """
    with open(filename, "r") as f:
        rows, cols, veh, n_rides, bonus, steps = map(int, f.readline().split())
        rides = []
        for _ in range(n_rides):
            a, b, x, y, s, f_end = map(int, f.readline().split())
            rides.append((a, b, x, y, s, f_end))
    return rows, cols, veh, n_rides, bonus, steps, rides


def manhattan(a, b):
    """Compute Manhattan distance between two points a and b."""
    return abs(a[0] - b[0]) + abs(a[1] - b[1])


def dp_single_vehicle(rides, bonus, steps, start_pos=(0, 0)):
    """
    Dynamic programming solver for one vehicle using Weighted Interval Scheduling.
    Each ride i: (a, b, x, y, s, f)
    Returns: max_score, list of chosen ride indices (local indices).
    """
    n = len(rides)
    if n == 0:
        return 0, []

    # Precompute ride properties
    ride_len = [manhattan((a, b), (x, y)) for (a, b, x, y, s, f) in rides]
    finish_time = [r[5] for r in rides]

    # Sort rides by finish time
    indexed_rides = sorted(enumerate(rides), key=lambda x: x[1][5])
    idx_map = [i for i, _ in indexed_rides]
    rides = [r for _, r in indexed_rides]
    finish_time = [r[5] for r in rides]

    # Precompute earliest possible start & finish times (from current pos = (0,0))
    earliest_finish = []
    for i in range(n):
        a, b, x, y, s, f = rides[i]
        go = manhattan(start_pos, (a, b))
        start_time = max(s, go)
        finish = start_time + ride_len[i]
        earliest_finish.append(finish)

    # Binary search helper: find last ride that finishes before current ride starts
    def find_compatible(i):
        a, b, x, y, s, f = rides[i]
        # we assume vehicle starts from origin for simplicity
        return bisect.bisect_right(finish_time, s - 1) - 1

    dp = [0] * (n + 1)
    parent = [-1] * n

    for i in range(n):
        a, b, x, y, s, f = rides[i]
        length = ride_len[i]
        go = manhattan(start_pos, (a, b))
        start = max(s, go)
        finish = start + length

        if finish > f or finish > steps:
            dp[i + 1] = dp[i]  # skip infeasible
            continue

        bonus_here = bonus if go <= s else 0
        score_i = length + bonus_here

        j = find_compatible(i)
        if j >= 0:
            option = dp[j + 1] + score_i
        else:
            option = score_i

        if option > dp[i]:
            dp[i + 1] = option
            parent[i] = j
        else:
            dp[i + 1] = dp[i]

    # reconstruct solution
    chosen = []
    i = n - 1
    while i >= 0:
        if parent[i] != -1 and dp[i + 1] > dp[i]:
            chosen.append(i)
            i = parent[i]
        else:
            i -= 1

    chosen.reverse()
    chosen_global_indices = [idx_map[c] for c in chosen]
    return dp[n], chosen_global_indices


def solve(rows, cols, veh, n_rides, bonus, steps, rides):
    """
    Multi-vehicle scheduler using DP per vehicle.
    """
    total_score = 0
    sched = [[] for _ in range(veh)]
    taken = [False] * n_rides

    for v in range(veh):
        # Filter available rides for this iteration
        available_rides = [(i, r) for i, r in enumerate(rides) if not taken[i]]
        if not available_rides:
            break

        ride_list = [r for _, r in available_rides]
        index_map = [i for i, _ in available_rides]

        score, chosen_local = dp_single_vehicle(ride_list, bonus, steps)
        total_score += score

        for local_idx in chosen_local:
            global_idx = index_map[local_idx]
            taken[global_idx] = True
            sched[v].append(global_idx)

    return total_score, sched


def write_output(filename, sched):
    """Write the vehicle schedules to output file."""
    with open(filename, "w") as f:
        for rides in sched:
            f.write(str(len(rides)) + " " + " ".join(map(str, rides)) + "\n")

    print("=== Vehicle Ride Assignments ===")
    for i, rides in enumerate(sched):
        print(f"Vehicle {i}: {len(rides)} rides -> {rides}")


if __name__ == "__main__":
    filename = "./input/group_C_instance.in"
    rows, cols, veh, n_rides, bonus, steps, rides = read_input(filename)

    score, sched = solve(rows, cols, veh, n_rides, bonus, steps, rides)
    print("Total Score =", score)

    os.makedirs("./output", exist_ok=True)
    base = os.path.basename(filename).replace(".in", ".out")
    outname = os.path.join("./output", base)
    write_output(outname, sched)
    print(f"\nOutput saved to {outname}")


Total Score = 41046031
=== Vehicle Ride Assignments ===
Vehicle 0: 93 rides -> [3464, 31, 4108, 6256, 2412, 4741, 4462, 1473, 4944, 6271, 6380, 5886, 2007, 4981, 4024, 1944, 1304, 2097, 6236, 6668, 267, 5603, 3393, 3177, 4637, 1207, 1953, 4868, 5924, 1722, 6701, 5849, 4293, 2117, 4347, 5869, 4379, 2994, 5099, 3995, 317, 853, 4144, 5743, 5253, 1830, 3348, 2014, 5795, 2511, 4716, 4928, 1873, 3028, 4888, 6443, 5139, 5521, 3602, 3375, 5087, 5059, 2038, 1157, 131, 5870, 2143, 21, 4556, 3121, 3343, 3921, 5266, 3652, 1819, 5278, 220, 3758, 3610, 2370, 2905, 132, 899, 5963, 374, 1256, 3425, 4964, 2773, 795, 2609, 5901, 1933]
Vehicle 1: 56 rides -> [3772, 2881, 3581, 5063, 93, 1257, 3984, 3752, 2603, 701, 3523, 5265, 410, 364, 1524, 6058, 5200, 1607, 5474, 1272, 2025, 4303, 1895, 3811, 578, 1199, 2856, 5854, 2116, 6200, 5118, 1943, 6422, 3216, 3742, 903, 3953, 1778, 5475, 4950, 955, 6698, 6203, 3519, 3487, 5725, 2542, 2640, 5393, 3529, 309, 2330, 593, 6638, 4971, 4537]
Vehicle 2: 45 rides -> [3

In [ ]:
import random

def generate_random_instance(filename, rows=3, cols=4, vehicles=2, n_rides=3,
                                              bonus=2, steps=10, seed=None):
    """
    Generate a random instance and print a human-readable description of rides.
    """
    if seed is not None:
        random.seed(seed)

    rides = []
    for i in range(n_rides):
        a = random.randint(0, rows-1)
        b = random.randint(0, cols-1)
        x = random.randint(0, rows-1)
        y = random.randint(0, cols-1)
        while a == x and b == y:
            x = random.randint(0, rows-1)
            y = random.randint(0, cols-1)

        earliest_start = random.randint(0, steps//2)
        ride_duration = abs(a - x) + abs(b - y)
        latest_finish = earliest_start + ride_duration + random.randint(0, steps//2)
        latest_finish = min(latest_finish, steps)

        rides.append((a, b, x, y, earliest_start, latest_finish))

    # Write to file
    with open(filename, "w") as f:
        f.write(f"{rows} {cols} {vehicles} {n_rides} {bonus} {steps}\n")
        for ride in rides:
            f.write(" ".join(map(str, ride)) + "\n")

    # Print human-readable description
    print(f"{rows} rows, {cols} columns, {vehicles} vehicles, {n_rides} rides, {bonus} bonus and {steps} steps")
    for i, (a, b, x, y, s, f) in enumerate(rides):
        print(f"ride {i+1} from [{a}, {b}] to [{x}, {y}], earliest start {s}, latest finish {f}")

    return rides

def compute_max_possible_score(rides, bonus):
    """
    Compute the maximum possible score assuming:
    - Every ride is completed.
    - Every ride starts at its earliest start (gets bonus).
    """
    max_score = 0
    for a, b, x, y, s, f in rides:
        ride_len = abs(a - x) + abs(b - y)
        max_score += ride_len + bonus
    return max_score


rides = generate_random_instance("./input/random_instance.in", rows=800, cols=1000,
                                 vehicles=100, n_rides=300, bonus=25, steps=25000, seed=42)

max_score = compute_max_possible_score(rides, bonus=25)
print("Maximum possible theoretical score (all rides completed + all bonuses):", max_score)

800 rows, 1000 columns, 100 vehicles, 300 rides, 25 bonus and 25000 steps
ride 1 from [654, 114] to [25, 759], earliest start 4506, latest finish 9792
ride 2 from [228, 142] to [754, 104], earliest start 11087, latest finish 23786
ride 3 from [558, 89] to [604, 432], earliest start 520, latest finish 1397
ride 4 from [95, 223] to [238, 517], earliest start 9863, latest finish 10734
ride 5 from [574, 203] to [733, 665], earliest start 11490, latest finish 21039
ride 6 from [429, 225] to [459, 603], earliest start 4557, latest finish 5071
ride 7 from [777, 825] to [163, 714], earliest start 6924, latest finish 13223
ride 8 from [284, 159] to [220, 980], earliest start 5514, latest finish 8073
ride 9 from [94, 389] to [99, 367], earliest start 5635, latest finish 15553
ride 10 from [270, 826] to [44, 747], earliest start 7527, latest finish 16617
ride 11 from [127, 996] to [387, 80], earliest start 9044, latest finish 15023
ride 12 from [643, 633] to [370, 591], earliest start 3150, lates